In [58]:

from ssapp.models.NeuralNetworkModels.variational_autoencoder import VAE
from ssapp.data.AntennaDatasetLoaders import load_serialized_dataset
from ssapp.Utils import train_test_data_split
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
%matplotlib inline
%load_ext autoreload
%autoreload 2

dataset = load_serialized_dataset('CircularHornDataset1',extra_back_steps=1)

train_dataset, test_dataset = train_test_data_split(dataset)

config = {'latent_size': 1,
            'coder_channel_1': 16,
            'coder_channel_2': 32,
            'batch_size' : 2}


train_dataloader = DataLoader(train_dataset,batch_size=config['batch_size'])
test_dataloader = DataLoader(test_dataset,batch_size=config['batch_size'])

train_params, train_fields = next(iter(train_dataloader))


model = VAE(config)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:

optimizer = optim.Adam(model.parameters(), lr=4e-3)
criterion = nn.MSELoss()



In [65]:



EPOCHS=10
for epoch in range(EPOCHS):
    epoch_loss,epoch_rec_loss,epoch_kl_loss = (0,0,0)

    for train_params, train_fields in train_dataloader:

        train_fields= train_fields.float()

        optimizer.zero_grad()
        reconstruction = model(train_fields.float())
        rec_loss = criterion(reconstruction, train_fields)
        
        loss = rec_loss + model.kl

        loss.backward()

        epoch_rec_loss += rec_loss/len(train_fields)
        epoch_kl_loss += model.kl/len(train_fields)
        epoch_loss += loss/len(train_fields)

        optimizer.step()



    print("epoch : {}/{}, rec_loss = {:.9e}, kl_loss = {:.9e}".format(epoch + 1, EPOCHS, epoch_rec_loss,epoch_kl_loss))

epoch : 1/10, rec_loss = 1.903418091e+03, kl_loss = 6.635011597e+02
epoch : 2/10, rec_loss = 1.860978638e+03, kl_loss = 4.852287598e+02
epoch : 3/10, rec_loss = 1.852744263e+03, kl_loss = 4.852440491e+02


KeyboardInterrupt: 